In [7]:
from flask import Flask, request
from flask_cors import CORS, cross_origin
from flask_restful import Resource, Api
from json import dumps
from flask_jsonpify import jsonify
import json
import pymongo
import urllib
from collections import Counter
import requests
#import ssl

from flask import (Flask, make_response, render_template, redirect, request,url_for)

In [9]:
ISSUER = 'sample-auth-server'
app = Flask(__name__)
api = Api(app)
CORS(app)

client = pymongo.MongoClient("mongodb+srv://hemanth:"+ urllib.quote("Hemanth@123")+"@cluster0-59obx.mongodb.net/test?retryWrites=true&w=majority")

with open('public.pem', 'rb') as f:
    public_key = f.read()

@app.route('/login', methods = ['POST'])
def request_token():
    request=json.loads(str(request.data))
    response = checkCredentails(request)

#     contents = json.loads(r.text)
#     access_token = contents.get('access_token')

#   # Writes access token to the cookie
#     response = make_response(redirect(url_for('main')))
#     response.set_cookie('access_token', access_token)

    return response


@app.route('/authorize', methods = ['POST'])
def verify_access_token(access_token):
    try:
        decoded_token = jwt.decode(request.access_token.encode(), public_key,
                                   issuer = ISSUER,
                                   algorithm = 'RS256')
    except (jwt.exceptions.InvalidTokenError,
          jwt.exceptions.InvalidSignatureError,
          jwt.exceptions.InvalidIssuerError,
          jwt.exceptions.ExpiredSignatureError):
        return False

    return True

In [11]:
def checkCredentails(request):
    username=request.username
    password=request.password
    client_id=request.client_id
    client_secret=request.client_secret
    
    if None in [username, password, client_id, client_secret]:
        return json.dumps({
          "error": "invalid_request"
        }), 400

    if not authenticate_user_credentials(username, password):
        return json.dumps({
          "error": "access_denied"
        }), 401
  
    access_token = generate_access_token()
    return json.dumps({ 
    "access_token": access_token,
    "token_type": "JWT",
    "expires_in": LIFE_SPAN
    })

In [ ]:
def authenticate_user_credentials(username, password):
    db=client.credentisls
    form=db.userDetails.find_one({'userName': username,'password':password})
    if(form):
        return true
    return false